In [106]:
"""
Tarang TO DO

1. FIX GITIGNORE
    Operates locally.
""";

In [107]:
# Importing Webscraping, data managment and visualization libraries.

import pandas as pd, numpy as np, matplotlib.pyplot as plt, seaborn as sns, random

# Web–scraping & cleaning libs.
import requests
from bs4 import BeautifulSoup

# NLP Libs.
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
nltk.download('punkt')
nltk.download('stopwords')
import re

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import confusion_matrix, plot_confusion_matrix
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier,\
AdaBoostClassifier, VotingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier,\
ExtraTreesClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
#
import xgboost as xgb

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/christophervolpacchio/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/christophervolpacchio/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [108]:
""" This is how we will start to extract the lyrics from the song webpage.
"""

url = 'https://www.lyrics.com/lyric/31024783/The+Strokes/Heart+In+a+Cage'
req = requests.get(url)

soup = BeautifulSoup(req.text)
lyrics = soup.find("pre",{'id' : 'lyric-body-text'}).text
lyrics

"Well, I don't feel better when I'm fucking around\nAnd I don't write better when I'm stuck in the ground\nSo don't teach me a lesson 'cause I've already learned\nYeah, the sun will be shining and my children will burn\n\nOh, the heart beats in its cage\n\nI don't want what you want, I don't feel what you feel\nSee, I'm stuck in a city but I belong in a field\nYeah, we got left, left, left, left, left, left, left\nNow it's three in the morning and you're eating alone\n\nOh, the heart beats in its cage\n\nAll our friends, they're laughing at us\nAll of those you loved, you mistrust\nHelp me, I'm just not quite myself\nLook around, there's no one else left\n\nI went to the concert and I fought through the crowd\nGuess I got too excited when I thought you were around\nOh, he gets left, left, left, left, left, left, left\nAnd I'm sorry you were thinking I would steal your fire\n\nOh, the heart beats in its cage\nYes, the heart beats in its cage\n\nAnd the heart beats in its cage"

In [109]:
# \n is a line \n\n is a stanza.
#<div class="clearfix"><h3 class="artist-album-label">

## Make df of lyrics and write to csv for initial NLP model dev. ?

In [110]:
lyrics 
#lyrics.to_csv('./capstone/data/test_lyrics.csv')

"Well, I don't feel better when I'm fucking around\nAnd I don't write better when I'm stuck in the ground\nSo don't teach me a lesson 'cause I've already learned\nYeah, the sun will be shining and my children will burn\n\nOh, the heart beats in its cage\n\nI don't want what you want, I don't feel what you feel\nSee, I'm stuck in a city but I belong in a field\nYeah, we got left, left, left, left, left, left, left\nNow it's three in the morning and you're eating alone\n\nOh, the heart beats in its cage\n\nAll our friends, they're laughing at us\nAll of those you loved, you mistrust\nHelp me, I'm just not quite myself\nLook around, there's no one else left\n\nI went to the concert and I fought through the crowd\nGuess I got too excited when I thought you were around\nOh, he gets left, left, left, left, left, left, left\nAnd I'm sorry you were thinking I would steal your fire\n\nOh, the heart beats in its cage\nYes, the heart beats in its cage\n\nAnd the heart beats in its cage"

In [111]:
temp_df = pd.DataFrame(lyrics.split('\n'))
# add stanza number, song name

def stanzas(df):
    counter = 1
    stanzas = []
    for row in df:
        if row != '':
            stanzas.append(counter)
        else:
            stanzas.append(0)
            counter +=1
    return stanzas
            

In [112]:
stanza_cnt = pd.DataFrame(stanzas(temp_df[0]))
stanza_cnt.head(2)
test = temp_df.merge(stanza_cnt, how = 'left', left_on = temp_df.index, right_on = stanza_cnt.index, left_index=True)
test.to_csv('./data/test_song.csv')

In [113]:
test.head()

,key_0,0_x,0_y
0,0,"Well, I don't feel better when I'm fucking around",1
1,1,And I don't write better when I'm stuck in the...,1
2,2,So don't teach me a lesson 'cause I've already...,1
3,3,"Yeah, the sun will be shining and my children ...",1
4,4,,0


In [114]:
test1 = test.copy()
test1 = test1[test1['0_y']!=0]
test1

,key_0,0_x,0_y
0,0,"Well, I don't feel better when I'm fucking around",1
1,1,And I don't write better when I'm stuck in the...,1
2,2,So don't teach me a lesson 'cause I've already...,1
3,3,"Yeah, the sun will be shining and my children ...",1
5,5,"Oh, the heart beats in its cage",2
7,7,"I don't want what you want, I don't feel what ...",3
8,8,"See, I'm stuck in a city but I belong in a field",3
9,9,"Yeah, we got left, left, left, left, left, lef...",3
10,10,Now it's three in the morning and you're eatin...,3
12,12,"Oh, the heart beats in its cage",4


In [ ]:
"""
Est. API querying; append to empty lists for lines?
Remove stopwords/ est stopwords to start?
Where to use randomizer?
Words coming up with null result will be skipped/ignored (i.e. slang)
Can use cvec to det. new song name? Or can use randomizer from top?

"""

In [39]:
"""
SHOULD I USE SUB FUNCTIONS
""";
user_id = open('./data/lyrics_user.txt', mode = 'r').read()
token = open('./data/lyrics_key.txt', mode = 'r').read()

artist = input('Enter artist here : ')
# if not redirected to artist page (instead results of ARTISTS, return message to include articles in name)
url = 'https://www.lyrics.com/artist/'+str(artist.replace(' ', '+'))
req = requests.get(url)
soup = BeautifulSoup(req.text)
tables = soup.find_all('table', {'class': 'tdata'})
songs = []
for table in tables:
    for row in table.find('tbody').find_all('tr'):
        songs.append(row.find('a').text)
songs = pd.DataFrame(set(songs), columns = ['Titles'])
print(songs)
# how to display FULL disc on Web Pge w/0 elipsis?
album = input('Choose a title by index number : ')
song = str(songs.loc[int(album),'Titles'])
#
# Getting album url to pull tracklist and search lyrics for each.
#
url = 'https://www.stands4.com/services/v2/lyrics.php'
params = {'uid' : str(user_id),
         'tokenid' : str(token),
         'term' : str(song),
         'format': 'json'}
req = requests.get(url, params)
req.json()
#
# Log album name and use url from dict to nav to page and pull track list for further querying
#

Enter artist here : the strokes
                             Titles
0                 Partners in Crime
1    Why Are Sunday's So Depressing
2                The End Has No End
3                   Heart in a Cage
4             12:51 [Album Version]
..                              ...
100               On the Other Side
101                         Someday
102                 Ode to the Mets
103                  Automatic Stop
104                        Selfless

[105 rows x 1 columns]
Choose a title by index number : 69


{'result': [{'song': 'Barely Legal',
   'song-link': 'https://www.lyrics.com/lyric/4691583/The+Strokes/Barely+Legal',
   'artist': 'The Strokes',
   'artist-link': 'https://www.lyrics.com/artist/The+Strokes/478260',
   'album': 'The Modern Age [UK CD]',
   'album-link': 'https://www.lyrics.com/album/529462/The Modern Age [UK CD]'},
  {'song': 'Barely Legal',
   'song-link': 'https://www.lyrics.com/lyric/4997595/The+Strokes/Barely+Legal',
   'artist': 'The Strokes',
   'artist-link': 'https://www.lyrics.com/artist/The+Strokes/478260',
   'album': 'Is This It [UK]',
   'album-link': 'https://www.lyrics.com/album/550860/Is This It [UK]'},
  {'song': 'Barely Legal',
   'song-link': 'https://www.lyrics.com/lyric/4997605/The+Strokes/Barely+Legal',
   'artist': 'The Strokes',
   'artist-link': 'https://www.lyrics.com/artist/The+Strokes/478260',
   'album': 'The Modern Age [US CD]',
   'album-link': 'https://www.lyrics.com/album/534632/The Modern Age [US CD]'},
  {'song': 'Barely Legal',
   's

In [43]:
artist

'the strokes'

In [47]:
"""
INITIAL ARTIST PAGE QUERY
"""


res = req.json()
df = res['result']
df
new_df = {}
for i in df:
    for key, value in i.items():
        if value.lower() == artist.lower():
            new_df.update(i)
            break
print(new_df)

"""
How to deal w/ duplicative results on an EP?
    Attribution for albums 'studio', etc.?
""";

{'song': 'Barely Legal', 'song-link': 'https://www.lyrics.com/lyric/35821913/The+Strokes/Barely+Legal', 'artist': 'The Strokes', 'artist-link': 'https://www.lyrics.com/artist/The+Strokes/478260', 'album': 'Is This It [180 Gram Vinyl]', 'album-link': 'https://www.lyrics.com/album/2367673/Is This It [180 Gram Vinyl]'}


In [ ]:
"""
Return tracklist from album page
go to track page & extract lyrics for each
dev DF & attribution
model
pos. sentiment analysis from meta returned from mw dic.?
option for 'clean' versions?


MAYBE two options: new song/clean song??
"""

In [46]:
album = input('Choose a title by index number : ')
str(songs.loc[int(album),'Titles'])

Choose a title by index number : 6


'New York City Cops'

In [41]:
# only works for scalars
#songs.at(5, 'Titles')

TypeError: '_AtIndexer' object is not callable

In [8]:
token = pd.read_csv('./data/lyrics_key.txt', )
print(token)

Empty DataFrame
Columns: [5fGYJwlI810Y5kTs]
Index: []


In [6]:
""" Use this to start to document attributes of music related to song req.
We can store information about the album, artist and song name while navigating to the album page.
From album page we can then loop through song tags and request to their respective pages. Lyric Query is above.
"""

# FN passing through index # of desired song. will work off assumption that fav song will be on album w/ sim tracks

user_id = pd.read_csv('./data/lyrics_user.txt')
token = pd.read_csv('./data/lyrics_key.txt')
song = 'Selfless'

url = 'https://www.stands4.com/services/v2/lyrics.php'
params = {'uid' : str(user_id),
         'tokenid' : str(token),
         'term' : str(song),
         'format': 'json'}
req = requests.get(url, params)
req.json()

#### Should be sub fn using specified artist to refine results to single dict. w/ key:val pairs to document & nav.

{'error': 'Invalid Token'}

----
----
---

### MW API

---

In [64]:
thesarus_key = open('./data/mw_key.txt', mode = 'r').read()
words = ['nice']
params = {
    'key' : thesarus_key
         }
for word in words:
    mw_url = 'https://www.dictionaryapi.com/api/v3/references/thesaurus/json/'
    mw_url += word
    req = requests.get(mw_url, params)
    res = req.json()
    print(res)
    
    #     df = res['meta']
#     df
#     new_df = {}
#     for i in df:
#         for key, value in i.items():
#             if value.lower() == artist.lower():
#                 new_df.update(i)
#                 break
#     print(new_df)

[{'meta': {'id': 'nice', 'uuid': '50503c08-17e7-45dd-8793-c97212a16ee8', 'src': 'coll_thes', 'section': 'alpha', 'target': {'tuuid': '8f3deb8e-d83b-4154-8e37-47587b5e4d84', 'tsrc': 'collegiate'}, 'stems': ['nice', 'nicely', 'niceness', 'nicenesses', 'nicer', 'nicest'], 'syns': [['befitting', 'correct', 'de rigueur', 'decent', 'decorous', 'genteel', 'polite', 'proper', 'respectable', 'seemly'], ['agreeable', 'blessed', 'congenial', 'darling', 'delectable', 'delicious', 'delightful', 'delightsome', 'dreamy', 'dulcet', 'enjoyable', 'felicitous', 'good', 'grateful', 'gratifying', 'heavenly', 'jolly', 'luscious', 'palatable', 'pleasant', 'pleasing', 'pleasurable', 'pretty', 'satisfying', 'savory', 'sweet', 'tasty', 'welcome'], ['choosy', 'dainty', 'delicate', 'demanding', 'exacting', 'fastidious', 'finical', 'finicking', 'finicky', 'fussbudgety', 'fussy', 'old-maidish', 'particular', 'pernickety', 'persnickety', 'picky'], ['affable', 'agreeable', 'amiable', 'genial', 'good-natured', 'good-t

In [65]:
res[0].keys()

dict_keys(['meta', 'hwi', 'fl', 'def', 'shortdef'])

In [66]:
res[0]['def']

[{'sseq': [[['sense',
     {'sn': '1',
      'dt': [['text',
        'following the established traditions of refined society and good taste '],
       ['vis', [{'t': 'had {it}nice{/it} manners'}]]],
      'syn_list': [[{'wd': 'befitting'},
        {'wd': 'correct'},
        {'wd': 'de rigueur'},
        {'wd': 'decent'},
        {'wd': 'decorous'},
        {'wd': 'genteel'},
        {'wd': 'polite'},
        {'wd': 'proper'},
        {'wd': 'respectable'},
        {'wd': 'seemly'}]],
      'rel_list': [[{'wd': 'acceptable'},
        {'wd': 'adequate'},
        {'wd': 'satisfactory'},
        {'wd': 'tolerable'}],
       [{'wd': 'dress'}, {'wd': 'dressy'}, {'wd': 'formal'}],
       [{'wd': 'dignified'}, {'wd': 'elegant'}, {'wd': 'gracious'}],
       [{'wd': 'priggish'}, {'wd': 'prim'}, {'wd': 'stiff'}, {'wd': 'stuffy'}],
       [{'wd': 'apt'}, {'wd': 'material'}, {'wd': 'relevant'}],
       [{'wd': 'compatible'}, {'wd': 'congenial'}, {'wd': 'harmonious'}],
       [{'wd': 'allowed'},
  

In [87]:
# https://dictionaryapi.com/products/json#sec-2.labels
#res[0]['fl'] --> word type

syns = []
for i in res[0]['def']:
    for n in i['sseq']:
        for x in n:
            
            for y in x:
                if type(y) == dict:
                    #print(y.keys())
                    for i in y['syn_list']:
                        for j['wd'] in i:
                            syns.append(j['wd'])

In [117]:
syns = pd.DataFrame(syns)['wd']
syns.head()

0     befitting
1       correct
2    de rigueur
3        decent
4      decorous
Name: wd, dtype: object

In [122]:
a = 'This is a line in a song using the word nice'
a.replace('nice', random.choice(list(syns)))

'This is a line in a song using the word tasty'

In [69]:
y['syn_list']

[[{'wd': 'ceremonious'},
  {'wd': 'correct'},
  {'wd': 'decorous'},
  {'wd': 'formal'},
  {'wd': 'proper'},
  {'wd': 'punctilious'},
  {'wd': 'starchy'},
  {'wd': 'stiff'},
  {'wd': 'stiff-necked'},
  {'wd': 'stilted'}]]

In [51]:
test = ['a','b','c']
for i in test:
    print(test.index(i))

0
1
2


In [29]:
get_meta(example)

https://www.dictionaryapi.com/api/v3/references/thesaurus/json/umpire?key=6d207136-9763-48c2-b218-6369e35e2036
https://www.dictionaryapi.com/api/v3/references/thesaurus/json/pussy?key=6d207136-9763-48c2-b218-6369e35e2036


In [ ]:
req = requests.get(mw_url, params)

In [ ]:
print(req.url)

In [ ]:
req.json()